In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#only include relevant columns
rel_col = ['idVisit', 'visitorId', 'visitorType', 'referrerType', 'country', 'type', 'url', 'pageIdAction', 'idpageview', 'serverTimePretty', 'timeSpent', 'pageviewPosition', 'title', 'timestamp', 'dimension1', 'dimension3', 'productViewPrice', 'productViewName', 'productViewSku', 'productViewCategories', 'eventCategory', 'eventAction', 'eventName', 'siteSearchKeyword', 'orderId', 'revenue', 'itemDetails','filter','filter_name','cat', 'cat2', 'cat3']
action_snippet = pd.read_csv("../data_exploration/data_snippets/cluster_sample.csv", sep=';', usecols = rel_col)

In [3]:
action_snippet = action_snippet.loc[action_snippet['country'] == 'Austria']
print(len(action_snippet))
action_snippet.drop_duplicates(inplace=True)
print(len(action_snippet))

columns_to_exclude = ['timestamp', 'timeServerPretty']
action_snippet.drop_duplicates(subset=action_snippet.columns.difference(columns_to_exclude), inplace=True)
print(len(action_snippet))


30206190
30206190
30206190


In [14]:
action_snippet.to_csv('../data_exploration/data_snippets/cluster_sample.csv', sep=";", index=False)

In [15]:
print(f"The sample contains {len(action_snippet['idVisit'].unique())} unique visits")
print(f"The sample contains {len(action_snippet['visitorId'].unique())} unique visitors")
print(f"They used {len(action_snippet['dimension3'].unique())} different filter settings")
print(f"They used {len(action_snippet['filter'].unique())} different filters")
unique_visitors = action_snippet.drop_duplicates(subset='visitorId')
print(f"Distribution of user types {unique_visitors['visitorType'].value_counts()} ")

The sample contains 3451551 unique visits
The sample contains 1709443 unique visitors
They used 85780 different filter settings
They used 10124 different filters
Distribution of user types visitorType
new                  1019841
returning             534225
returningCustomer     155377
Name: count, dtype: int64 


In [4]:
#only keep relevant rows to trace a user session
#keep all page views (type= "action")
#keep only the events with the eventName "Filter", eventAction containing "offer_click", evenAction "Add.onWishlist", 
#keep all ecommerce orders
conditions = ((action_snippet['type'] == 'action') | 
               (action_snippet['type'] == 'search') | 
               (action_snippet['type'] == 'ecommerceOrder')| 
               ((action_snippet['type'] == 'event') & 
                ((action_snippet['eventAction'].str.contains('offer_click')) | 
                 (action_snippet['eventName'] == 'Add.onWishlist')|
                 (action_snippet['eventName'] == 'search.submit')|
                 (action_snippet['eventName'] == 'searchvariant')|
                 (action_snippet['eventName'] == 'searcha')|
                 (action_snippet['eventName'] == 'searchfs')|
                 (action_snippet['eventName'] == 'searchcat')|
                (action_snippet['dimension1'] == 'Produktvergleich')|
                (action_snippet['dimension1'] == 'Deals')| 
                 (action_snippet['eventName'] == 'Filter'))))
#apply the conditions to filter the DataFrame
filtered_snippet = action_snippet[conditions]
#del action_snippet
filtered_snippet = filtered_snippet.drop_duplicates() 

In [17]:
filtered_snippet.to_csv("../data_exploration/data_snippets/filtered_sample.csv", sep=";", index=False)

In [2]:
filtered_snippet = pd.read_csv("../data_exploration/data_snippets/filtered_sample.csv", sep=';')

In [5]:
#define aggregation functions
agg_funcs = {
    
    'visitorId': 'first',
    'visitorType': 'first',
    'referrerType': 'first',
    'pageviewPosition': lambda x: x.max(),  #count number of page views
    'cat2': lambda x: x.nunique(),  #count number of unique second level categories visited
    'cat3': lambda x: x.nunique(),  #count number of unique third level categories visited
    'productViewSku': lambda x: x.nunique(),  #count number of unique productViewSku
    'orderId': lambda x: x.notnull().sum(),  #count number of leads (ecommerceOrder)
    'timeSpent': 'sum',
    'eventName': lambda x: (x == 'Filter').sum(),  #count number of filters applied
    'revenue': lambda x: x.sum(),  #sum of revenue
    'cat': lambda x: x.value_counts().idxmax() if not x.dropna().empty else None, #classify the visit by most visited category
    'dimension1' : lambda x:  (x == 'Produktvergleich').sum()

}

#aggregate statistics by idVisit
visit_statistics = filtered_snippet.groupby('idVisit').agg(agg_funcs)
#define aggregation function for counting different 'search' events 
visit_statistics['nddsearches'] = filtered_snippet['eventName'].isin(['searchvariant', 'searcha', 'searchfs', 'searchcat']).groupby(filtered_snippet['idVisit']).sum()

#compute additional statistics
visit_statistics['nsearches'] = filtered_snippet[filtered_snippet['type'] == 'search'].groupby('idVisit').size()
visit_statistics['dealActions'] = filtered_snippet[filtered_snippet['dimension1'] == 'Deals'].groupby('idVisit').size()
visit_statistics['nwishlist'] = filtered_snippet[filtered_snippet['eventName'] == 'Add.onWishlist'].groupby('idVisit').size()
visit_statistics['meanPageviewTime'] = visit_statistics['timeSpent'].div(visit_statistics['pageviewPosition']).replace(np.inf, 0)

# Rename columns for clarity
visit_statistics.rename(columns={
    'pageviewPosition': 'npageViews',
    'dimension1': 'productCompareActions',
    'productViewSku': 'nproducts',
    'orderId': 'nleads',
    'eventName': 'nfilters',
    'revenue': 'leadSum',
    'cat' : 'category',
    'cat2' : '2ndLevelVisits',
    'cat3' : '3rdLevelVisits'
}, inplace=True)
visit_statistics['avgLeadSum'] = visit_statistics['leadSum']/visit_statistics['nproducts']
#reset index to make idVisit a regular column
visit_statistics.reset_index(inplace=True)

#if all entries in leadSum are NaN, replace NaNs with 0
visit_statistics = visit_statistics.fillna(0)


In [6]:
visit_statistics.to_csv("../data_exploration/data_snippets/visit_statistics.csv", sep=";", index=False)

In [5]:
visit_statistics = pd.read_csv('../data_exploration/data_snippets/visit_statistics.csv', sep=';')


In [7]:
visit_statistics['category'].value_counts()

category
Hardware                 1146519
0                         846779
Haushalt                  354740
Telefon                   281914
Video, Foto & TV          204679
Baumarkt & Garten         152239
Sport & Freizeit          103330
Unsortiert                 67288
Drogerie                   64585
Audio & HiFi               52856
Games                      39842
Spielzeug & Modellbau      39838
Auto & Motorrad            31895
Büro & Schule               7157
Software                    6509
Filme                       1459
Name: count, dtype: int64

In [4]:
visit_statistics = visit_statistics.loc[visit_statistics['npageViews'] > 0]
#visit_statistics = visit_statistics.loc[visit_statistics['timeSpent'] > 0]

In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

visit_statistics.describe(percentiles = [0.01,0.02,0.03,0.04,0.05,0.25,0.5,0.75,0.95,0.96,0.97,0.98,0.99]).T

,count,mean,std,min,1%,2%,3%,4%,5%,25%,50%,75%,95%,96%,97%,98%,99%,max
idVisit,3401629.000,256566863.495,13701188.118,233146321.000,233304248.120,233479410.120,234696373.040,234827563.480,235455147.000,245326684.000,256692659.000,268515917.000,279430969.400,279577047.560,279718449.480,279858522.200,280281451.440,280430512.000
npageViews,3401629.000,5.270,11.170,1.000,1.000,1.000,1.000,1.000,1.000,1.000,2.000,5.000,19.000,22.000,26.000,33.000,49.000,500.000
2ndLevelVisits,3401629.000,0.881,0.871,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,2.000,2.000,2.000,3.000,4.000,48.000
3rdLevelVisits,3401629.000,0.914,1.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,2.000,2.000,3.000,3.000,4.000,74.000
nproducts,3401629.000,1.403,3.637,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,4.000,5.000,6.000,8.000,11.000,500.000
nleads,3401629.000,0.418,1.072,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,2.000,3.000,3.000,5.000,68.000
timeSpent,3401629.000,252.156,666.942,0.000,0.000,0.000,0.000,0.000,0.000,0.000,18.000,158.000,1434.000,1622.000,1839.000,2244.000,3038.000,86307.000
nfilters,3401629.000,0.580,2.544,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,5.000,6.000,8.000,11.000,257.000
leadSum,3401629.000,218.684,1438.329,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1128.400,1379.000,1741.962,2368.740,3743.000,977064.000
productCompareActions,3401629.000,0.079,1.146,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,365.000


In [24]:
print(f"The visitors performed {visit_statistics['nsearches'].sum()} searches")
print(f"The visitors performed {visit_statistics['nsearches'].mean()} mean searches")
print(f"The visitors performed {visit_statistics['nsearches'].median()} median searches")
print(f"Total page views {visit_statistics['npageViews'].sum()}")
print(f"The visitors visited {visit_statistics['npageViews'].mean()} mean pages")
print(f"The visitors visited {visit_statistics['npageViews'].median()} median pages")
#print(f"Total category visits {visit_statistics['categoriesVisited'].sum()}")
#print(f"The visitors visited {visit_statistics['categoriesVisited'].mean()} mean categories")
#print(f"The visitors visited {visit_statistics['categoriesVisited'].median()} median categories")
print(f"The visitors used {visit_statistics['nfilters'].sum()} filters")
print(f"The visitors used {visit_statistics['nfilters'].mean()} mean filters")
print(f"The visitors used {visit_statistics['nfilters'].median()} median filters")
print(f"The visitors added {visit_statistics['nwishlist'].sum()} products to the wishlist")
print(f"The visitors added {visit_statistics['nwishlist'].mean()} mean products to the wishlist")
print(f"The visitors added {visit_statistics['nwishlist'].median()} median products to the wishlist")
print(f"The visitors generated {visit_statistics['nleads'].sum()} leads")
print(f"The visitors generated {visit_statistics['nleads'].mean()} mean leads")
print(f"The visitors generated {visit_statistics['nleads'].median()} median leads")
print(f"The visitors generated a total revenue of {visit_statistics['leadSum'].sum()}")
print(f"The visitors generated a mean revenue of {visit_statistics['leadSum'].mean()}")
print(f"The visitors generated a median revenue of {visit_statistics['leadSum'].median()}")
print(f"The visitors spent {visit_statistics['timeSpent'].sum()/60} minutes on Geizhals")
print(f"The visitors spent mean {visit_statistics['timeSpent'].mean()/60} minutes on Geizhals")
print(f"The visitors spent median {visit_statistics['timeSpent'].median()/60} minutes on Geizhals")

The visitors performed 2299903.0 searches
The visitors performed 0.6761181187013634 mean searches
The visitors performed 0.0 median searches
Total page views 17927213.0
The visitors visited 5.270184667404941 mean pages
The visitors visited 2.0 median pages
The visitors used 1973291 filters
The visitors used 0.5801017688877887 mean filters
The visitors used 0.0 median filters
The visitors added 789.0 products to the wishlist
The visitors added 0.0002319476932963589 mean products to the wishlist
The visitors added 0.0 median products to the wishlist
The visitors generated 1423431 leads
The visitors generated 0.41845568696645047 mean leads
The visitors generated 0.0 median leads
The visitors generated a total revenue of 743882351.0500001
The visitors generated a mean revenue of 218.6841513433711
The visitors generated a median revenue of 0.0
The visitors spent 14295700.416666666 minutes on Geizhals
The visitors spent 4.20260422775872 minutes on Geizhals
The visitors spent 0.3 minutes on G

In [25]:
#explore extreme values
for column in visit_statistics.columns:
    if visit_statistics[column].dtype != object:
        top_20_rows = visit_statistics.nlargest(20, column)
        print(f"Top 20 rows with the highest values in column '{column}':")
        print(top_20_rows)
        print("\n")

Top 20 rows with the highest values in column 'idVisit':
           idVisit         visitorId        visitorType referrerType  \
3401628  280430512  c74e518d17935432                new       search   
3401627  280430511  101291b8301d13c9                new       search   
3401626  280430510  ce46861e054ac5d0  returningCustomer       search   
3401625  280430508  4f45ff9ae8ee163e          returning       direct   
3401624  280430502  061e055ff802f230                new       direct   
3401623  280430498  51d00b86ff20f048  returningCustomer       direct   
3401622  280430497  aaa737cf65c93e89  returningCustomer       direct   
3401621  280430494  b6ba567d78142cdf                new       search   
3401620  280430491  0cf376248ec0abc4          returning       direct   
3401619  280430485  430658841ebc838d                new       search   
3401618  280430483  d773fd61e3b55734          returning      website   
3401617  280430476  00a4edceda2d4b51                new       direct   
3401616

In [26]:
#there are some sessions that appear to be scraping the site; sessions are just subsequent page visits with a roughly equal duration of page visits and no actions in between
#filter all sessions with 5 or more page visits
sessions_with_time = action_snippet.dropna(subset=['timeSpent']).groupby('idVisit').filter(lambda x: len(x) >= 5)

#define a function to check if values are approximately equal within a tolerance
def approximately_equal(values, tolerance=2):
    return all(abs(values - values.iloc[0]) <= tolerance)

#group by idVisit and apply the function to filter sessions with consistent timeSpent values
unique_sessions = sessions_with_time.groupby('idVisit').filter(lambda x: approximately_equal(x['timeSpent']))
unique_session_ids = unique_sessions['idVisit'].unique()
irregular_sessions = action_snippet[action_snippet['idVisit'].isin(unique_session_ids)]
#conclusion: bots/scrapers avoid this as there are some page visits with varying timeSpent; this approach mostly filters short sessions

KeyboardInterrupt: 

In [ ]:
#inspect example visits with high numbers of searches
inspect1 = action_snippet[action_snippet['idVisit'] == 251465075]
inspect2 = action_snippet[action_snippet['idVisit'] == 280218642]

In [ ]:
#inspect visits with no timeSpent on page
inspect3 = visit_statistics[visit_statistics['timeSpent'] ==0]
inspect3 = inspect3['idVisit'].unique()
inspect3 = action_snippet[action_snippet['idVisit'].isin(inspect3)]

In [27]:
visit_statistics.columns  

Index(['idVisit', 'visitorId', 'visitorType', 'referrerType', 'npageViews',
       '2ndLevelVisits', '3rdLevelVisits', 'nproducts', 'nleads', 'timeSpent',
       'nfilters', 'leadSum', 'category', 'productCompareActions',
       'nddsearches', 'nsearches', 'dealActions', 'nwishlist',
       'meanPageviewTime', 'timeSpentSum'],
      dtype='object')

In [ ]:
sum_columns = ['npageViews',
       '2ndLevelVisits', '3rdLevelVisits', 'nproducts', 'nleads', 'timeSpent',
       'nfilters', 'leadSum', 'category', 'productCompareActions',
       'nddsearches', 'nsearches', 'dealActions', 'nwishlist',
       'meanPageviewTime', 'timeSpentSum']

#sum visit statistics by user
summed_statistics = visit_statistics.groupby('visitorId')[sum_columns].sum().reset_index()

In [ ]:
summed_statistics.describe()

,npageViews,2ndLevelVisits,3rdLevelVisits,nproducts,nleads,timeSpent,nfilters,leadSum,productCompareActions,nddsearches,nsearches,dealActions,nwishlist,meanPageviewTime
count,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000,1699960.000
mean,10.546,1.764,1.828,2.807,0.837,504.566,1.161,437.588,0.159,0.454,1.353,0.357,0.000,94.824
std,45.269,5.597,6.241,18.882,3.003,2662.258,7.979,2955.191,3.782,3.810,7.176,7.999,0.034,353.282
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,3.000,1.000,1.000,1.000,0.000,34.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,15.000
75%,7.000,2.000,2.000,2.000,1.000,263.000,0.000,112.993,0.000,0.000,1.000,0.000,0.000,65.000
max,17024.000,1197.000,1505.000,16913.000,967.000,634289.000,1522.000,977064.000,3031.000,2131.000,2578.000,3515.000,16.000,75660.075


In [ ]:
print((visit_statistics[sum_columns] == 0).all(axis=1).sum())

0
